In [120]:
import pandas as pd
df=pd.read_csv('data/uec_data_2023_2025_full.csv')

### Filter out the Totals
They are inaccurate given the potental negatives!!
Totals were calculated after

In [121]:
df_filtered = df[~df["Hospital"].str.endswith(" Total")]

#### Map the Hospitals to regions

Region map!

In [122]:
hospital_to_region_dict = {
    # HSE Dublin and North East
    "Beaumont Hospital":                        "HSE Dublin and North East",
    "Cavan General Hospital":                   "HSE Dublin and North East",
    "Connolly Hospital":                        "HSE Dublin and North East",
    "Louth County Hospital":                    "HSE Dublin and North East",
    "Mater Misericordiae University Hospital":  "HSE Dublin and North East",
    "National Orthopaedic Hospital Cappagh":    "HSE Dublin and North East",
    "Our Lady of Lourdes Hospital":             "HSE Dublin and North East",
    "Our Lady's Hospital Navan":                "HSE Dublin and North East",

    # HSE Dublin and Midlands
    "CHI at Crumlin":                           "HSE Dublin and Midlands",
    "CHI at Tallaght":                          "HSE Dublin and Midlands",
    "CHI at Temple Street":                     "HSE Dublin and Midlands",
    "MRH Mullingar":                            "HSE Dublin and Midlands",
    "MRH Portlaoise":                           "HSE Dublin and Midlands",
    "MRH Tullamore":                            "HSE Dublin and Midlands",
    "Naas General Hospital":                    "HSE Dublin and Midlands",
    "St. James's Hospital":                     "HSE Dublin and Midlands",
    "St. Luke's Radiation Oncology Network":    "HSE Dublin and Midlands",
    "Tallaght University Hospital":             "HSE Dublin and Midlands",

    # HSE Dublin and South East
    "National Rehabilitation Hospital":         "HSE Dublin and South East",
    "St. Columcille's Hospital":                "HSE Dublin and South East",
    "St Luke's General Hospital Kilkenny":     "HSE Dublin and South East",
    "St. Michael's Hospital":                   "HSE Dublin and South East",
    "St. Vincent's University Hospital":        "HSE Dublin and South East",
    "Tipperary University Hospital":            "HSE Dublin and South East",
    "UH Waterford":                             "HSE Dublin and South East",
    "Wexford General Hospital":                 "HSE Dublin and South East",

    # HSE South West
    "Bantry General Hospital":                      "HSE South West",
    "Cork University Hospital":                     "HSE South West",
    "Mallow General Hospital":                      "HSE South West",
    "Mercy University Hospital":                    "HSE South West",
    "South Infirmary Victoria University Hospital": "HSE South West",
    "UH Kerry":                                     "HSE South West",

    # HSE Mid West
    "Ennis Hospital":                               "HSE Mid West",
    "Nenagh Hospital":                              "HSE Mid West",
    "St. John's Hospital Limerick":                 "HSE Mid West",
    "UH Limerick":                                  "HSE Mid West",

    # HSE West and North West
    "Galway University Hospital":                   "HSE West and North West",
    "Letterkenny University Hospital":              "HSE West and North West",
    "Mayo University Hospital":                     "HSE West and North West",
    "Portiuncula University Hospital":              "HSE West and North West",
    "Roscommon University Hospital":                "HSE West and North West",
    "Sligo University Hospital":                    "HSE West and North West"
}

#### Map hospitals to regions
In a new column

In [123]:
# map hospitals to regions in new column
df_filtered['Region'] = df_filtered['Hospital'].map(hospital_to_region_dict)

/var/folders/8p/bnwtr4b93fn3_th1x9ysmwy00000gn/T/ipykernel_91971/3362337087.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Region'] = df_filtered['Hospital'].map(hospital_to_region_dict)


Empty total values

In [124]:
df_filtered[df_filtered['Total'].isna()]['Hospital'].value_counts()

Hospital
National Orthopaedic Hospital Cappagh           716
St. Luke's Radiation Oncology Network           716
National Rehabilitation Hospital                716
St. Columcille's Hospital                       716
St. Michael's Hospital                          716
Bantry General Hospital                         716
Mallow General Hospital                         716
South Infirmary Victoria University Hospital    716
Ennis Hospital                                  716
Nenagh Hospital                                 716
St. John's Hospital Limerick                    716
Roscommon University Hospital                   716
Louth County Hospital                           673
UH Waterford                                    128
Wexford General Hospital                          1
Name: count, dtype: int64

Negative values

In [125]:
df_filtered[df_filtered['Total']<0]['Hospital']

Series([], Name: Hospital, dtype: object)

Check Empty Hosptials

In [126]:
df_filtered.groupby('Hospital')['Total'].sum().sort_values(ascending=True).head(20)

Hospital
Bantry General Hospital                            0.0
St. Michael's Hospital                             0.0
St. Luke's Radiation Oncology Network              0.0
St. John's Hospital Limerick                       0.0
St. Columcille's Hospital                          0.0
South Infirmary Victoria University Hospital       0.0
Roscommon University Hospital                      0.0
Nenagh Hospital                                    0.0
National Rehabilitation Hospital                   0.0
UH Waterford                                       0.0
Mallow General Hospital                            0.0
National Orthopaedic Hospital Cappagh              0.0
Louth County Hospital                              0.0
Ennis Hospital                                     0.0
MRH Tullamore                                     56.0
CHI at Tallaght                                 1022.0
Connolly Hospital                               1148.0
Our Lady's Hospital Navan                       1426.0
C

Save csv!
* All Hospitals
* With Regions

In [127]:
# Save it to a csv
df_filtered.to_csv('data/uec_data_2023_2025_full_with_regions.csv')

## Total populations

In [128]:
# Load the filtered data back in to verify
df = pd.read_csv('data/uec_data_2023_2025_full_with_regions.csv')

Make Weekday column

In [129]:

df['weekday']= pd.to_datetime(df['Date'], dayfirst=True).dt.day_of_week
# Convert Date to datetime
df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

# Then sort by date to ensure proper order
df = df.sort_values('Date')


Make Encatchment df

In [130]:
# get encatchment and turn into a dict
pop_df=pd.read_csv("data/encatchment_areas.csv")
pop_dict=dict(zip(pop_df.iloc[:, 0], pop_df.iloc[:, 1]))
# map Region -> population into new column 'Region_Population'
df['Region_Population'] = df['Region'].map(pop_dict)
# quick check of unmapped region values
#print("Unmapped region values:", df[df['Region_Population'].isna()]['Region'].dropna().unique())

In [131]:
pop_dict

{'HSE Dublin and North East': 1187082,
 'HSE Dublin and Midlands': 1077639,
 'HSE Dublin and South East': 971093,
 'HSE South West': 740614,
 'HSE Mid West': 413059,
 'HSE West and North West': 759652}

## Weight totals
Rate per 10k people
$$\frac{df_{total\ trollies}}{df_{region\ population}}\cdot 10000$$

In [132]:
df['Trollies Per 10k']=(df['Total']/df['Region_Population'])*10000

## Daily 
long and wide dfs

In [133]:
df_wide=df.groupby(['Date','Region'])['Trollies Per 10k'].sum().unstack()

df_long = df_wide.reset_index().melt(
    id_vars="Date",
    var_name="Region",
    value_name="Value"
)

Save

In [134]:
df_wide.to_csv("data/wide_daily_scaledPer10k.csv")
df_long.to_csv("data/long_daily_scaledPer10k.csv")

# Weekly
Long and wide dfs

In [135]:
df_week_long=df_long.groupby(['Region', pd.Grouper(key='Date', freq='W')]).sum()
df_week_wide=df_week_long.pivot_table(index='Region', columns='Date', values='Value')

Save

In [136]:
df_week_long.to_csv("data/long_weekly_scaledPer10k.csv")
df_week_wide.to_csv("data/wide_weekly_scaledPer10k.csv")